In [1]:
import numpy as np
import lightgbm as lgb
from wideboost.wrappers import wlgb
from sklearn import datasets

In [2]:
digits = datasets.load_digits()

train_idx = np.random.choice(np.arange(1797),round(1797*0.7),replace=False)
test_idx = np.setdiff1d(np.arange(1797),train_idx)


xtrain = digits['data']
ytrain = digits['target']


train_data = lgb.Dataset(xtrain[train_idx,:], label=ytrain[train_idx])
test_data = lgb.Dataset(xtrain[test_idx,:], label=ytrain[test_idx])

In [3]:
from hyperopt import fmin, tpe, hp, STATUS_OK, space_eval

best_val = 1.0

def objective(param):
    global best_val
    #watchlist = [(dtrain,'train'),(dtest,'test')]
    ed1_results = dict()
    print(param)
    param['num_leaves'] = round(param['num_leaves']+1)
    param['min_data_in_leaf'] = round(param['min_data_in_leaf'])
    wbst = lgb.train(param,
                      train_data,
                      num_boost_round=100,
                      valid_sets=test_data,
                      evals_result=ed1_results)
    output = min(ed1_results['valid_0']['multi_error'])
    
    if output < best_val:
        print("NEW BEST VALUE!")
        best_val = output
    
    return {'loss': output, 'status': STATUS_OK }

spc = {
    'objective': hp.choice('objective',['multiclass']),
    'metric':hp.choice('metric',['multi_error']),
    'num_class':hp.choice('num_class',[10]),
    'learning_rate': hp.loguniform('learning_rate', -7, 0),
    'num_leaves' : hp.qloguniform('num_leaves', 0, 7, 1),
    'feature_fraction': hp.uniform('feature_fraction', 0.5, 1),
    'bagging_fraction': hp.uniform('bagging_fraction', 0.5, 1),
    'min_data_in_leaf': hp.qloguniform('min_data_in_leaf', 0, 6, 1),
    'min_sum_hessian_in_leaf': hp.loguniform('min_sum_hessian_in_leaf', -16, 5),
    'lambda_l1': hp.choice('lambda_l1', [0, hp.loguniform('lambda_l1_positive', -16, 2)]),
    'lambda_l2': hp.choice('lambda_l2', [0, hp.loguniform('lambda_l2_positive', -16, 2)])
}


best = fmin(objective,
    space=spc,
    algo=tpe.suggest,
    max_evals=100)

{'bagging_fraction': 0.7101086524260212, 'feature_fraction': 0.6287739686374094, 'lambda_l1': 0, 'lambda_l2': 0.6257014876978995, 'learning_rate': 0.07268396291983138, 'metric': 'multi_error', 'min_data_in_leaf': 6.0, 'min_sum_hessian_in_leaf': 2.008724606685398e-05, 'num_class': 10, 'num_leaves': 76.0, 'objective': 'multiclass'}
[1]	valid_0's multi_error: 0.183673                    
[2]	valid_0's multi_error: 0.111317                    
[3]	valid_0's multi_error: 0.0871985                   
[4]	valid_0's multi_error: 0.0742115                   
[5]	valid_0's multi_error: 0.0723562                   
[6]	valid_0's multi_error: 0.0630798                   
[7]	valid_0's multi_error: 0.0575139                   
[8]	valid_0's multi_error: 0.0630798                   
[9]	valid_0's multi_error: 0.0630798                   
[10]	valid_0's multi_error: 0.0630798                  
[11]	valid_0's multi_error: 0.0649351                  
[12]	valid_0's multi_error: 0.0630798               

[23]	valid_0's multi_error: 0.435993                                              
[24]	valid_0's multi_error: 0.437848                                              
[25]	valid_0's multi_error: 0.42115                                               
[26]	valid_0's multi_error: 0.42115                                               
[27]	valid_0's multi_error: 0.41744                                               
[28]	valid_0's multi_error: 0.415584                                              
[29]	valid_0's multi_error: 0.426716                                              
[30]	valid_0's multi_error: 0.423006                                              
[31]	valid_0's multi_error: 0.424861                                              
[32]	valid_0's multi_error: 0.419295                                              
[33]	valid_0's multi_error: 0.413729                                              
[34]	valid_0's multi_error: 0.410019                                              
[35]

[17]	valid_0's multi_error: 0.183673                                              
[18]	valid_0's multi_error: 0.183673                                              
[19]	valid_0's multi_error: 0.187384                                              
[20]	valid_0's multi_error: 0.185529                                              
[21]	valid_0's multi_error: 0.179963                                              
[22]	valid_0's multi_error: 0.178108                                              
[23]	valid_0's multi_error: 0.176252                                              
[24]	valid_0's multi_error: 0.174397                                              
[25]	valid_0's multi_error: 0.174397                                              
[26]	valid_0's multi_error: 0.166976                                              
[27]	valid_0's multi_error: 0.165121                                              
[28]	valid_0's multi_error: 0.168831                                              
[29]

[11]	valid_0's multi_error: 0.571429                                              
[12]	valid_0's multi_error: 0.554731                                              
[13]	valid_0's multi_error: 0.504638                                              
[14]	valid_0's multi_error: 0.491651                                              
[15]	valid_0's multi_error: 0.482375                                              
[16]	valid_0's multi_error: 0.474954                                              
[17]	valid_0's multi_error: 0.469388                                              
[18]	valid_0's multi_error: 0.460111                                              
[19]	valid_0's multi_error: 0.463822                                              
[20]	valid_0's multi_error: 0.45269                                               
[21]	valid_0's multi_error: 0.443414                                              
[22]	valid_0's multi_error: 0.441558                                              
[23]

[5]	valid_0's multi_error: 0.758813                                               
[6]	valid_0's multi_error: 0.717996                                               
[7]	valid_0's multi_error: 0.667904                                               
[8]	valid_0's multi_error: 0.636364                                               
[9]	valid_0's multi_error: 0.632653                                               
[10]	valid_0's multi_error: 0.564007                                              
[11]	valid_0's multi_error: 0.538033                                              
[12]	valid_0's multi_error: 0.495362                                              
[13]	valid_0's multi_error: 0.486085                                              
[14]	valid_0's multi_error: 0.48423                                               
[15]	valid_0's multi_error: 0.480519                                              
[16]	valid_0's multi_error: 0.474954                                              
[17]

...

[76]	valid_0's multi_error: 0.176252                                                
[77]	valid_0's multi_error: 0.172542                                                
[78]	valid_0's multi_error: 0.172542                                                
[79]	valid_0's multi_error: 0.172542                                                
[80]	valid_0's multi_error: 0.172542                                                
[81]	valid_0's multi_error: 0.170686                                                
[82]	valid_0's multi_error: 0.168831                                                
[83]	valid_0's multi_error: 0.170686                                                
[84]	valid_0's multi_error: 0.168831                                                
[85]	valid_0's multi_error: 0.170686                                                
[86]	valid_0's multi_error: 0.168831                                                
[87]	valid_0's multi_error: 0.168831                             

[68]	valid_0's multi_error: 0.0445269                                               
[69]	valid_0's multi_error: 0.0408163                                               
[70]	valid_0's multi_error: 0.0426716                                               
[71]	valid_0's multi_error: 0.0408163                                               
[72]	valid_0's multi_error: 0.0408163                                               
[73]	valid_0's multi_error: 0.0408163                                               
[74]	valid_0's multi_error: 0.0408163                                               
[75]	valid_0's multi_error: 0.0408163                                               
[76]	valid_0's multi_error: 0.0408163                                               
[77]	valid_0's multi_error: 0.038961                                                
[78]	valid_0's multi_error: 0.038961                                                
[79]	valid_0's multi_error: 0.038961                             

[60]	valid_0's multi_error: 0.0333952                                               
[61]	valid_0's multi_error: 0.0333952                                               
[62]	valid_0's multi_error: 0.0333952                                               
[63]	valid_0's multi_error: 0.0333952                                               
[64]	valid_0's multi_error: 0.0333952                                               
[65]	valid_0's multi_error: 0.0333952                                               
[66]	valid_0's multi_error: 0.0333952                                               
[67]	valid_0's multi_error: 0.0333952                                               
[68]	valid_0's multi_error: 0.0333952                                               
[69]	valid_0's multi_error: 0.0333952                                               
[70]	valid_0's multi_error: 0.0315399                                               
[71]	valid_0's multi_error: 0.0315399                            

[52]	valid_0's multi_error: 0.918367                                                
[53]	valid_0's multi_error: 0.918367                                                
[54]	valid_0's multi_error: 0.918367                                                
[55]	valid_0's multi_error: 0.918367                                                
[56]	valid_0's multi_error: 0.918367                                                
[57]	valid_0's multi_error: 0.918367                                                
[58]	valid_0's multi_error: 0.918367                                                
[59]	valid_0's multi_error: 0.918367                                                
[60]	valid_0's multi_error: 0.918367                                                
[61]	valid_0's multi_error: 0.918367                                                
[62]	valid_0's multi_error: 0.918367                                                
[63]	valid_0's multi_error: 0.918367                             

[44]	valid_0's multi_error: 0.133581                                                
[45]	valid_0's multi_error: 0.12616                                                 
[46]	valid_0's multi_error: 0.12616                                                 
[47]	valid_0's multi_error: 0.12616                                                 
[48]	valid_0's multi_error: 0.124304                                                
[49]	valid_0's multi_error: 0.122449                                                
[50]	valid_0's multi_error: 0.120594                                                
[51]	valid_0's multi_error: 0.118738                                                
[52]	valid_0's multi_error: 0.120594                                                
[53]	valid_0's multi_error: 0.122449                                                
[54]	valid_0's multi_error: 0.120594                                                
[55]	valid_0's multi_error: 0.111317                             

In [4]:
print(best_val)
print(space_eval(spc, best))

0.016697588126159554
{'bagging_fraction': 0.7330846139995021, 'feature_fraction': 0.9603469330267634, 'lambda_l1': 0, 'lambda_l2': 0, 'learning_rate': 0.34266101559119244, 'metric': 'multi_error', 'min_data_in_leaf': 19.0, 'min_sum_hessian_in_leaf': 2.1110575117798897e-05, 'num_class': 10, 'num_leaves': 8.0, 'objective': 'multiclass'}


In [6]:
1 - 0.0037 / 0.01669

0.7783103654883163